In [2]:
from collections import Counter
from itertools import combinations
from math import factorial,ceil

## SON

In [3]:
a = "37000337270"
(13-len(a))*"0"+a

'0037000337270'

In [143]:
l = [('100', {'2', '13', '12', '1', '16'}), ('98', {'10', '18', '5', '3', '13', '12', '1', '8', '9', '16', '11', '19'}), ('101', {'13', '2', '7', '4', '12', '1', '6', '16', '15'}), ('102', {'3'
, '13', '4', '12', '1', '8', '6', '19'}), ('99', {'18', '17', '3', '13', '2', '7', '4', '8', '9', '16', '11', '15'}), ('97', {'10', '18', '14', '5', '17', '3', '2', '7', '4', '8', '9', '11',
 '15', '19'}), ('103', {'4', '8', '3', '13'}), ('105', {'12', '3', '13'}), ('104', {'8'}), ('108', {'12', '13'}), ('106', {'12', '13'}), ('107', {'12', '13'})]

In [176]:
cb = lambda n,k: factorial(n)/(factorial(k)*factorial(n - k))

In [68]:
def multi_hash(part):
    #pass 1
    item_counter = Counter([ele for tup in part for ele in tup[1]])
    item_size = len(item_counter)
    pair_size = int(item_size*(item_size-1)/2)
    bucket_num = int(pair_size/3)+1
    hash_1 = lambda x: hash(max(x)+min(x))%bucket_num
    hash_2 = lambda x: hash(min(x)+max(x))%bucket_num
    bit_map_1 = {i:0 for i in range(bucket_num)}
    bit_map_2 = {i:0 for i in range(bucket_num)}
    for tup in part:
        pairs = combinations(tup[1],2)
        for pair in pairs:
            bit_map_1[hash_1(pair)] += 1
            bit_map_2[hash_2(pair)] += 1

    print(bit_map_1)
    print(bit_map_2)
    bit_map_1 = {key:1 if value>= thres else 0 for key,value in bit_map_1.items()}
    bit_map_2 = {key:1 if value>= thres else 0 for key,value in bit_map_2.items()}


    #pass 2
    table = [[{i,j},0] for i,j in combinations(item_counter.keys(),2)\
            if (item_counter[i]>=thres) and (item_counter[j]>=thres)\
            and bit_map_1[hash_1({i,j})]==1\
            and bit_map_2[hash_2({i,j})]==1]
    for tup in part:
        temp = tup[1]
        for row in table:
            i,j = row[0]
            if i in temp and j in temp:
                row[1] += 1
    return [row[0] for row in table if row[1]>=thres]

In [167]:
def apriori(part):
    freq_itsets = []
    baskets = [tup[1] for tup in part]
    N = max([len(x) for x in baskets])
    item_counter = Counter([ele for tup in baskets for ele in tup])
    prev_freq_itsets = [{ele} for ele,value in item_counter.items() if value>=thres]
    freq_itsets += prev_freq_itsets
    for n in range(2,N+1):
        # Construct
        item_counter = []
        for i,j in combinations(prev_freq_itsets,2):
            if len(i&j)==(n-2):
                row = [i.union(j),0]
                if row not in item_counter:
                    item_counter.append(row)
        print(len(item_counter))
        # Count
        for row in item_counter:
            itemset = row[0]
            for basket in baskets:
                if itemset.issubset(basket):
                    row[1] += 1
        prev_freq_itsets = [row[0] for row in item_counter if row[1]>=thres]
        freq_itsets += prev_freq_itsets
        if prev_freq_itsets == []:
            break
    return freq_itsets
    

In [155]:
def gen_bit_maps(baskets, bucket_num, hash_1, hash_2, n):
    bit_map_1 = {i:0 for i in range(bucket_num)}
    bit_map_2 = {i:0 for i in range(bucket_num)}
    for basket in baskets:
        combs = combinations(basket,n)
        for itset in combs:
            bit_map_1[hash_1(itset)] += 1
            bit_map_2[hash_2(itset)] += 1
    bit_map_1 = {key:1 if value>= thres else 0 for key,value in bit_map_1.items()}
    bit_map_2 = {key:1 if value>= thres else 0 for key,value in bit_map_2.items()}
    return bit_map_1, bit_map_2

In [ ]:
435 989 134 6

In [172]:
def multi_hash(part):
    freq_itsets = []
    baskets = [tup[1] for tup in part]
    N = max([len(x) for x in baskets])
    L = len(baskets)
    item_counter = Counter([ele for tup in baskets for ele in tup])
    prev_freq_itsets = [{ele} for ele,value in item_counter.items() if value>=thres]
    freq_itsets += prev_freq_itsets
    bucket_num = int(2/3*L*cb(N,2)/thres)+2
    hash_1 = lambda x: 2*hash("".join(set(x)))%bucket_num
    hash_2 = lambda x: hash("".join(set(x)))%bucket_num
    bit_map_1, bit_map_2 = gen_bit_maps(baskets, bucket_num, hash_1, hash_2, 2)
    for n in range(2,N+1):
        # Construct
        item_counter = []
        for i,j in combinations(prev_freq_itsets,2):
            if len(i&j)==(n-2):
                temp = i.union(j)
                row = [temp,0]
                if row not in item_counter and bit_map_1[hash_1(temp)] and bit_map_2[hash_2(temp)]:
                    item_counter.append(row)
        print(len(item_counter))
        # Count
        for row in item_counter:
            itemset = row[0]
            for basket in baskets:
                if itemset.issubset(basket):
                    row[1] += 1
        prev_freq_itsets = [row[0] for row in item_counter if row[1]>=thres]
        freq_itsets += prev_freq_itsets
        if prev_freq_itsets == []:
            break
        # BitMap
        bucket_num = int(cb(N,n+1)/2)+2
        bit_map_1, bit_map_2 = gen_bit_maps(baskets, bucket_num, hash_1, hash_2, n+1)
    return freq_itsets
    

In [168]:
apriori(l)

1
0


[{'12'}, {'13'}, {'12', '13'}]

In [177]:
multi_hash(l)

1
0


[{'12'}, {'13'}, {'12', '13'}]

In [70]:
def multi_hash(part):
    #pass 1
    item_counter = Counter([ele for tup in part for ele in tup[1]])
    item_size = len(item_counter)
    pair_size = int(item_size*(item_size-1)/2)
    bucket_num = int(pair_size/3)+1
    hash_1 = lambda x: hash(max(x)+min(x))%bucket_num
    hash_2 = lambda x: hash(min(x)+max(x))%bucket_num
    bit_map_1 = Counter([hash_1(pair) for tup in part for pair in combinations(tup[1],2)])
    bit_map_2 = Counter([hash_2(pair) for tup in part for pair in combinations(tup[1],2)])
    print(bit_map_1)
    print(bit_map_2)
    bit_map_1 = {key:1 if value>= thres else 0 for key,value in bit_map_1.items()}
    bit_map_2 = {key:1 if value>= thres else 0 for key,value in bit_map_2.items()}


    #pass 2
    table = [[{i,j},0] for i,j in combinations(item_counter.keys(),2)\
            if (item_counter[i]>=thres) and (item_counter[j]>=thres)\
            and bit_map_1[hash_1({i,j})]==1\
            and bit_map_2[hash_2({i,j})]==1]
    for tup in part:
        temp = tup[1]
        for row in table:
            i,j = row[0]
            if i in temp and j in temp:
                row[1] += 1
    return [row[0] for row in table if row[1]>=thres]

In [71]:

multi_hash(l)

Counter({40: 17, 10: 12, 51: 12, 20: 11, 24: 11, 0: 10, 3: 10, 17: 10, 49: 10, 29: 9, 13: 9, 23: 9, 22: 9, 21: 8, 43: 8, 55: 8, 25: 8, 19: 7, 15: 7, 44: 7, 12: 7, 31: 6, 14: 6, 57: 6, 11: 6, 26: 6, 4: 5, 50: 5, 34: 5, 48: 5, 56: 5, 45: 5, 28: 4, 33: 4, 2: 4, 9: 4, 41: 3, 16: 3, 54: 3, 36: 2, 47: 2, 6: 2, 37: 2, 35: 2, 46: 2, 27: 2, 52: 2, 42: 2, 53: 2, 1: 1, 8: 1, 5: 1, 32: 1, 39: 1})
Counter({49: 16, 46: 15, 0: 15, 33: 11, 30: 11, 20: 11, 2: 9, 35: 9, 54: 9, 27: 9, 3: 9, 56: 8, 44: 8, 31: 8, 42: 8, 34: 8, 24: 7, 9: 7, 51: 7, 37: 6, 5: 6, 16: 6, 41: 6, 38: 6, 19: 6, 47: 6, 1: 6, 23: 5, 28: 5, 45: 5, 6: 4, 39: 4, 14: 4, 52: 4, 36: 4, 4: 4, 26: 4, 55: 4, 12: 3, 32: 3, 53: 2, 40: 2, 15: 2, 18: 2, 25: 2, 48: 2, 13: 2, 10: 2, 11: 2, 7: 1, 17: 1, 43: 1, 22: 1, 21: 1})


[{'12', '13'}, {'13', '3'}, {'3', '8'}]

In [6]:
{key:1 if value>= thres else 0 for key,value in bit_map_1.items()}

{'12': 0, '23f': 1}

In [4]:
Counter([ele for pair in l for ele in pair[1]])

Counter({'12': 2, '124': 2, '15': 2, '11': 2, '126': 1})

In [44]:
s = {'12','15','124','14','125','235'}
len(list(combinations(s,2)))

15

In [8]:
hash(min({'asdfwqefq2','f423f-g'})+max({'asdfwqefq2','f423f-g'}))

1308689722708787155

In [9]:
hash(max({'asdfwqefq2','f423f-g'})+min({'asdfwqefq2','f423f-g'}))

-7768537218748576398

In [1]:
hash(("asdf","45h"))

-7650997984214878628

In [4]:
set(('"4710011405133"', '"4710011406123"', '"4710011401135"', '"4710011401128"'))

{'"4710011401128"', '"4710011401135"', '"4710011405133"', '"4710011406123"'}

In [8]:
sorted(['"4710011406123"', '"4710011409056"', '"4710011401135"', '"4710011401128"'])

['"4710011401128"', '"4710011401135"', '"4710011406123"', '"4710011409056"']

In [8]:
for tup in l:
    pairs = combinations(tup[1],2)
    for pair in pairs:
        print(pair)
        print(hash(pair[0])%3)
        print(hash(pair[1])%10)

('12', '124')
1
9
('12', '15')
1
1
('124', '15')
1
1
('12', '124')
1
9
('12', '11')
1
7
('124', '11')
1
7
('11', '126')
1
6
('11', '15')
1
1
('126', '15')
0
1


## Preprocess

In [4]:
{1,2,4}.issubset({1,2,3})

False